# Investigating the State Anomaly Detection

In [35]:
import os
import sys
import torch
import numpy as np
import pandas as pd
from time import time
from scipy import stats
from tabulate import tabulate

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report, accuracy_score

In [2]:
os.getcwd()

'/Users/jankreischer/Library/Mobile Documents/com~apple~CloudDocs/Master-Thesis/Code/state_anomaly_detection'

In [4]:
os.chdir(".")

In [15]:
from enum import Enum

class Behavior(Enum):
    NORMAL = "normal"
    ROOTKIT_BDVL = "bdvl"
    ROOTKIT_BEURK = "beurk"
    CNC_BACKDOOR_JAKORITAR = "backdoor_jakoritar"
    CNC_THETICK = "the_tick"
    CNC_OPT1 = "data_leak_1"
    CNC_OPT2 = "data_leak_2"
    RANSOMWARE_POC = "ransomware_poc"

class MTDTechnique(Enum):
    CNC_IP_SHUFFLE = "cnc_ip_shuffle"
    ROOTKIT_SANITIZER = "rootkit_sanitizer"
    RANSOMWARE_DIRTRAP = "ransomware_directory_trap"
    RANSOMWARE_FILE_EXT_HIDE = "ransomware_file_extension_hide"
    
actions = [
    MTDTechnique.CNC_IP_SHUFFLE,
    MTDTechnique.ROOTKIT_SANITIZER,
    MTDTechnique.RANSOMWARE_DIRTRAP,
    MTDTechnique.RANSOMWARE_FILE_EXT_HIDE
]

In [16]:
relative_data_path = "data"

# These samples were collected without only the attack influencing the device running
# No extra MTD/Agent/Components were running on the respective ElectroSense device.
behavior_datasets_file_paths: dict[Behavior, str] = {
    Behavior.NORMAL: f"{relative_data_path}/normal_expfs_online_samples_1_2022-08-20-09-16_5s.csv",
    Behavior.RANSOMWARE_POC: f"{relative_data_path}/ransom_expfs_online_samples_1_2022-08-22-14-04_5s.csv",
    Behavior.ROOTKIT_BDVL: f"{relative_data_path}/rootkit_bdvl_online_samples_1_2022-08-19-08-45_5s.csv",
    Behavior.ROOTKIT_BEURK: f"{relative_data_path}/rootkit_beurk_online_samples_1_2022-09-01-18-12_5s.csv",
    Behavior.CNC_THETICK: f"{relative_data_path}/cnc_thetick_online_samples_1_2022-08-30-16-11_5s.csv",
    Behavior.CNC_BACKDOOR_JAKORITAR: f"{relative_data_path}/cnc_backdoor_jakoritar_new_online_samples_1_2022-09-02-09-19_5s.csv",
    Behavior.CNC_OPT1: f"{relative_data_path}/cnc_opt_1_file_extr_online_samples_1_2022-09-24-22-08_5s.csv",
    Behavior.CNC_OPT2: f"{relative_data_path}/cnc_opt_2_sysinfo_online_samples_1_2022-09-24-14-04_5s.csv",
}

In [17]:
# TODO: These columns are derived from data_availability.py -> check data

constant_columns = ['cpuNice', 'cpuHardIrq', 'alarmtimer:alarmtimer_fired', 'tasksStopped',
                    'alarmtimer:alarmtimer_start', 'cachefiles:cachefiles_create',
                    'cachefiles:cachefiles_lookup', 'cachefiles:cachefiles_mark_active',
                    'dma_fence:dma_fence_init', 'udp:udp_fail_queue_rcv_skb']
# suspected cyclic/unstable
correlated_features = ['tasks', 'tasksSleeping', 'tasksZombie', 'tasksRunning',
                   'ramFree', 'ramUsed', 'ramCache', 'memAvail', 'numEncrypted',
                   'iface0RX', 'iface0TX', 'iface1RX', 'iface1TX'
                   ]

# suspected undesirably reactive, distorting afterstates
correlated_features += ['cpuSystem', 'block:block_dirty_buffer', 'cpuSoftIrq', 'cs', 'cpu-migrations',
                    'irq:softirq_entry', 'kmem:kmem_cache_alloc', 'kmem:kmem_cache_free',
                    'random:urandom_read', 'raw_syscalls:sys_enter', 'raw_syscalls:sys_exit',
                    'sched:sched_switch', 'sched:sched_wakeup', 'skb:consume_skb', 'timer:hrtimer_start',
                    'writeback:global_dirty_state']

correlated_features += ['cpuIdle', 'cpuIowait', 'block:block_bio_backmerge', 'block:block_touch_buffer', 'clk:clk_set_rate',
                    'irq:irq_handler_entry', 'jbd2:jbd2_start_commit', 'kmem:mm_page_alloc', 'kmem:mm_page_free',
                    'preemptirq:irq_enable', 'sock:inet_sock_set_state']

In [18]:
time_status_columns = ["time", "timestamp", "seconds"]


def get_filtered_df(path, 
                    filter_suspected_external_events=True,
                    start_index=10,
                    end_index=-1,
                    filter_constant_columns=True,
                    filter_outliers=True,
                    filter_correlated_features=False):
    df = pd.read_csv(path)

    if filter_suspected_external_events:
        # filter first hour of samples: 3600s / 50s = 72
        # and drop last measurement due to the influence of logging in, respectively out of the server
        df = df.iloc[start_index:end_index]

    # filter for measurements where the device was connected
    df = df[df['connectivity'] == 1]

    # Remove model-irrelevant time status columns
    df = df.drop(time_status_columns, axis=1)

    if filter_outliers:
        # drop outliers per measurement, indicated by (absolute z score) > 3
        df = df[(np.nan_to_num(np.abs(stats.zscore(df))) < 3).all(axis=1)]
        
    if filter_constant_columns:
        df = df.drop(constant_columns, axis=1)

    if filter_correlated_features:
        df = df.drop(correlated_features, axis=1)

    assert df.isnull().values.any() == False, "behavior data should not contain NaN values"
    return df

In [19]:
def filter_train_split_for_outliers(df: pd.DataFrame, b: Behavior, split=0.8):
    df = df[df["attack"] == b].drop(["attack"], axis=1)
    df_test = df.sample(frac=1 - split)  # .reset_index(drop=True)
    df_train = pd.concat([df, df_test]).drop_duplicates(keep=False)
    train_filtered = df_train[(np.nan_to_num(np.abs(stats.zscore(df_train))) < 3).all(axis=1)]
    train_filtered["attack"] = b
    df_test["attack"] = b
    train_filtered = train_filtered.to_numpy()
    df_test = df_test.to_numpy()
    return train_filtered, df_test

In [26]:
def parse_no_mtd_behavior_data(filter_suspected_external_events=True,
                               filter_constant_columns=True,
                               filter_outliers=True,
                               filter_correlated_features=False,
                               ) -> dict[Behavior, np.ndarray]:
    # print(os.getcwd())
    full_df = pd.DataFrame()
    for attack in behavior_datasets_file_paths:
        print(f"getting {attack}")
        df = get_filtered_df(behavior_datasets_file_paths[attack],
                                            filter_suspected_external_events=filter_suspected_external_events,
                                            start_index=50,
                                            filter_constant_columns=filter_constant_columns,
                                            filter_outliers=filter_outliers,
                                            filter_correlated_features=filter_correlated_features)
        df['attack'] = attack
        full_df = pd.concat([full_df, df])

    return full_df

In [27]:
def get_scaled_train_test_split(split=0.8, scaling_minmax=True, scale_normal_only=True, filter_outliers=True):
    """
    Method returns dictionaries mapping behaviors to scaled train and test data, as well as the scaler used
    Either decision states or raw behaviors can be utilized (decision flag) as no combinations
    with mtd need to be considered
    """
    #print(os.getcwd())
    rdf = parse_no_mtd_behavior_data(filter_outliers=filter_outliers)
    print(f"type(rdf): {type(rdf)}")
    print(f"rdf.columns: {rdf.columns}; {len(rdf.columns)}")
    # take split of all behaviors, concat, calc scaling, scale both train and test split
    train_filtered, df_test = filter_train_split_for_outliers(rdf, Behavior.NORMAL, split)
    print(f"type(train_filtered): {type(train_filtered)}")
    print(f"type(df_test): {type(df_test)}")

    # get behavior dicts for train and test
    train_bdata = {}
    test_bdata = {}
    for b in rdf["attack"].unique():
        dtrain_filtered, df_test = filter_train_split_for_outliers(rdf, b, split)
        train_bdata[b] = dtrain_filtered
        test_bdata[b] = df_test
        if b != Behavior.NORMAL and not scale_normal_only:
            train_filtered = np.vstack((train_filtered, train_bdata[b]))

    # fit scaler on either just normal data (if scale_normal_only), or all training data combined
    scaler = StandardScaler() if not scaling_minmax else MinMaxScaler()
    print(f"Using scaler {scaler}")
    scaler.fit(train_filtered[:, :-1])

    # get behavior dicts for scaled train and test data
    scaled_train = {}
    scaled_test = {}
    for b, d in train_bdata.items():
        scaled_train[b] = np.hstack((scaler.transform(d[:, :-1]), np.expand_dims(d[:, -1], axis=1)))
        scaled_test[b] = np.hstack(
            (scaler.transform(test_bdata[b][:, :-1]), np.expand_dims(test_bdata[b][:, -1], axis=1)))

    # return also scaler in case of using the agent for online scaling
    return scaled_train, scaled_test, scaler

In [28]:
def split_data(data, split=0.8):
    row = int(len(data) * split)
    X_train = data[:row, :-1].astype(np.float32)
    X_valid = data[row:, :-1].astype(np.float32)
    return X_train, X_valid

In [29]:
import numpy as np

def get_test_dataset(test_data):
    test_data_dict = {}

    for behavior, behavior_data in test_data.items():
        if behavior == Behavior.NORMAL:
            behavior_data = behavior_data[:2800]
            #continue
        else:
            behavior_data = behavior_data[:400]

        test_data_dict[behavior] = behavior_data

    return test_data_dict

In [32]:
training_data, test_data, _ = get_scaled_train_test_split(scaling_minmax=True,
                                                                         scale_normal_only=True)
normal_data = training_data[Behavior.NORMAL]
threshold = int(len(normal_data) * 0.5)

training_data[Behavior.NORMAL] = normal_data[:threshold]

ae_training_data = normal_data[threshold:]  # use remaining samples for autoencoder
ae_training_x, ae_valid_x = split_data(ae_training_data)

N_FEATURES = normal_data.shape[1] -1
flattend_test_data = np.empty([0, N_FEATURES+1])
for behavior, behavior_data in test_data.items():
    if behavior == Behavior.NORMAL:
        NR_SAMPLES = 2800
        behavior_data[:, -1] =  0
    else:
        NR_SAMPLES = 400
        behavior_data[:, -1] = 1
    #y_true = np.array([0 if behavior == Behavior.NORMAL else 1] * NR_SAMPLES)
    
    flattend_test_data = np.concatenate((flattend_test_data, behavior_data[:NR_SAMPLES]), axis=0)

ae_test_x = flattend_test_data[:,:-1]
ae_test_y = flattend_test_data[:,-1].astype(int)

evaluation_data = {}
for behavior, behavior_data in training_data.items():
    if behavior == Behavior.NORMAL:
        evaluation_data[behavior] = behavior_data[:2800]
    else:
        evaluation_data[behavior] = behavior_data[:400]

getting Behavior.NORMAL
getting Behavior.RANSOMWARE_POC
getting Behavior.ROOTKIT_BDVL
getting Behavior.ROOTKIT_BEURK
getting Behavior.CNC_THETICK
getting Behavior.CNC_BACKDOOR_JAKORITAR
getting Behavior.CNC_OPT1
getting Behavior.CNC_OPT2
type(rdf): <class 'pandas.core.frame.DataFrame'>
rdf.columns: Index(['connectivity', 'cpuUser', 'cpuSystem', 'cpuIdle', 'cpuIowait',
       'cpuSoftIrq', 'tasks', 'tasksRunning', 'tasksSleeping', 'tasksZombie',
       'ramFree', 'ramUsed', 'ramCache', 'memAvail', 'iface0RX', 'iface0TX',
       'iface1RX', 'iface1TX', 'numEncrypted', 'block:block_bio_backmerge',
       'block:block_bio_remap', 'block:block_dirty_buffer',
       'block:block_getrq', 'block:block_touch_buffer', 'block:block_unplug',
       'clk:clk_set_rate', 'cpu-migrations', 'cs', 'fib:fib_table_lookup',
       'filemap:mm_filemap_add_to_page_cache', 'gpio:gpio_value',
       'ipi:ipi_raise', 'irq:irq_handler_entry', 'irq:softirq_entry',
       'jbd2:jbd2_handle_start', 'jbd2:jbd2_start

/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered["attack"] = b
/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered["attack"] = b


type(train_filtered): <class 'numpy.ndarray'>
type(df_test): <class 'numpy.ndarray'>


/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered["attack"] = b
/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered["attack"] = b
/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

Using scaler MinMaxScaler()


In [38]:
def initial_autoencoder_architecture(n_features):
    return nn.Sequential(
        nn.Linear(n_features, 23),
        nn.BatchNorm1d(23),
        nn.GELU(),
        nn.Linear(23, 11),
        nn.GELU(),
        nn.Linear(11, 23),
        nn.BatchNorm1d(23),
        nn.GELU(),
        nn.Linear(23, n_features,),
        nn.GELU()
    )

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        self.eps = eps
        
    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [43]:
class AutoEncoder(torch.nn.Module):
    

    def __init__(self, model, X_valid, X_test, y_test, evaluation_data, n_std=20, activation_function=torch.nn.ReLU(), batch_size: int = 64, verbose=False):

        super().__init__()
        
        validation_dataset = torch.utils.data.TensorDataset(
            torch.from_numpy(X_valid).type(torch.float),

        )
        self.validation_data_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=True, drop_last=True)

        self.X_test = X_test
        self.y_test = y_test
        
        self.evaluation_data = evaluation_data

        n_features = X_test.shape[1]
        
        self.model = model
        self.threshold = None
        self.loss_mean = None
        self.loss_standard_deviation = None
        
        self.verbose = verbose

        
    def forward(self, X):
        return self.model(X)
    
    
    def pretrain(self, X_train, optimizer=torch.optim.SGD, loss_function=torch.nn.MSELoss(reduction='sum'), num_epochs: int = 15, batch_size=64, verbose=False):
        
        training_dataset = torch.utils.data.TensorDataset(
            torch.from_numpy(X_train).type(torch.float),
        )
        training_data_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

        epoch_losses = []
        #for e in tqdm(range(num_epochs), unit="epoch", leave=False):
        for e in range(num_epochs):
            self.train()
            current_losses = []
            for batch_index, (inputs,) in enumerate(training_data_loader):
                optimizer.zero_grad()
                outputs = self.forward(inputs)
                loss = loss_function(inputs, outputs)
                loss.backward()
                optimizer.step()
                current_losses.append(loss.item())
            
            epoch_losses.append(np.average(current_losses))
            if verbose:
                print(f'Training Loss in epoch {e + 1}: {epoch_losses[e]}')
            
        self.analyze_loss()

    '''
    This function uses normal data samles 
    after training the autoencoder to determine
    values that can be considered normal
    for the reconstruction loss based on normal samples
    '''
    def analyze_loss(self):
        losses = []
        
        self.eval() 
        with torch.no_grad():
            loss_function = torch.nn.MSELoss(reduction='sum')
            for batch_index, (inputs,) in enumerate(self.validation_data_loader):
                outputs = self.forward(inputs)
                loss = loss_function(inputs, outputs)
                losses.append(loss.item())
        
        losses = np.array(losses)

        self.loss_mean = losses.mean()
        self.loss_standard_deviation = losses.std()

        
    def predict(self, x, n_std=3):
        test_data = torch.utils.data.TensorDataset(
            torch.from_numpy(x).type(torch.float32)
        )
        test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

        all_predictions = torch.tensor([])  # .cuda()

        self.eval()
        with torch.no_grad():
            ae_loss = torch.nn.MSELoss(reduction="sum")
            for idx, (batch_x,) in enumerate(test_data_loader):
                model_predictions = self.forward(batch_x)
                model_predictions = ae_loss(model_predictions, batch_x).unsqueeze(0)  # unsqueeze as batch_size set to 1
                all_predictions = torch.cat((all_predictions, model_predictions))

        threshold = self.loss_mean + n_std * self.loss_standard_deviation
        all_predictions = (all_predictions > threshold).type(torch.long)
        return all_predictions.flatten()
    
    
    def predict_deviation(self, x):
        test_data = torch.utils.data.TensorDataset(
            torch.from_numpy(x).type(torch.float32)
        )
        test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

        prediction_errors = torch.tensor([])
        loss_function = torch.nn.MSELoss(reduction="sum")
        
        self.eval()
        with torch.no_grad():
            
            for batch_index, (inputs,) in enumerate(test_data_loader):
                prediction = self.forward(inputs)
                prediction_error = loss_function(inputs, prediction).unsqueeze(0)  # unsqueeze as batch_size set to 1
                prediction_errors = torch.cat((prediction_errors, prediction_error))

        return prediction_errors
    
    
    def score(self):
        n_std, accuracy = self.accuracy_score(None, None)
        if self.verbose:
            print(f"Highest validation accuracy achieved {accuracy:.2f} with n_std={n_std}")
            self.evaluate(n_std)
        return accuracy
    
    
    def accuracy_score(self, X, y):
        #if not self.threshold:
        #loss_mean, loss_standard_deviation = self.analyze_loss(X)
        #n_stds = np.arange(0.1, 3, 0.1)
        if self.loss_mean == None or self.loss_standard_deviation == None:
              #print("accuracy_score_optimized > accurcy_loss()")
              self.analyze_loss()
    
        best_accuracy = 0
        best_n_std = 0
        #accuracies = []
        y_dev = self.predict_deviation((self.X_test).astype(np.float32))
        for n_std in self.n_stds:
            y_true = self.y_test
            threshold = self.loss_mean + n_std * self.loss_standard_deviation
            y_pred = (y_dev > threshold).type(torch.long).detach().cpu().numpy()
            
            accuracy = accuracy_score(y_true, y_pred)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_n_std = n_std
            #if self.verbose:
            #    print(f"n_std {n_std:.2f} -> accuracy: {accuracy}")

        return best_n_std, best_accuracy
    
    
    def evaluate(self, n_std=3, tablefmt='pipe'):
        results = []
        labels= [0,1]
        pos_label = 1
            
        y_true_total = np.empty([0])
        y_pred_total = np.empty([0])
        for behavior, data in self.evaluation_data.items():
            y_true = np.array([0 if behavior == Behavior.NORMAL else 1] * len(data)).astype(int)
            y_true_total = np.concatenate((y_true_total, y_true))
            
            print(f"Using n_std: {n_std} as prediction threshold")
            y_pred = self.predict(data[:, :-1].astype(np.float32), n_std=n_std)
            y_pred_total = np.concatenate((y_pred_total, y_pred))

            accuracy = accuracy_score(y_true, y_pred)

            n_samples = len(y_true)
            results.append([behavior.name.replace("_", "\_"), f'{(100 * accuracy):.2f}\%', '\\notCalculated', '\\notCalculated', '\\notCalculated', str(n_samples)])

        accuracy = accuracy_score(y_true_total, y_pred_total)
        precision = precision_score(y_true_total, y_pred_total, average='binary', labels=labels, pos_label=pos_label, zero_division=1)
        recall = recall_score(y_true_total, y_pred_total, average='binary', labels=labels, pos_label=pos_label, zero_division=1)
        f1 = f1_score(y_true_total, y_pred_total, average='binary', labels=labels, pos_label=pos_label, zero_division=1)
        n_samples = len(y_true_total)
        results.append(["GLOBAL", f'{(100 * accuracy):.2f}\%', f'{(100 * precision):.2f}\%', f'{(100 * recall):.2f}\%', f'{(100 * f1):.2f}\%', n_samples])
        print("-----------")
        print(tabulate(results, headers=["Behavior", "Accuracy", "Precision", "Recall", "F1-Score", "\#Samples"], tablefmt=tablefmt)) 

In [44]:
autoencoder = AutoEncoder(initial_autoencoder_architecture(N_FEATURES), ae_valid_x, ae_test_x, ae_test_y, evaluation_data)
autoencoder.pretrain(ae_training_x, optimizer=torch.optim.Adam(autoencoder.parameters(), lr=1e-4,  weight_decay=0.01), loss_function=RMSELoss(), num_epochs=100, batch_size=64, verbose=False)

In [49]:
autoencoder.evaluate(n_std=20, tablefmt='latex_raw')

Using n_std: 20 as prediction threshold
Using n_std: 20 as prediction threshold
Using n_std: 20 as prediction threshold
Using n_std: 20 as prediction threshold
Using n_std: 20 as prediction threshold
Using n_std: 20 as prediction threshold
Using n_std: 20 as prediction threshold
Using n_std: 20 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        2532 \\
 RANSOMWARE\_POC          & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &         400 \\
 ROOTKIT\_BDVL            & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &         400 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &         400 \\
 CNC\_THETICK             & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &    